# 1. Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

# 2. Load Data

In [2]:
train = pd.read_excel('Data/train.xlsx', index_col='index')
train_reviews = pd.read_excel('Data/train_reviews.xlsx', index_col='index')

# 3. Preprocessing (train.xlsx)

In [28]:
def lowercase(df):
    '''
    lowercases all the text in the dataframe
    '''
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x.lower() if type(x) == str else x)
    return df

def general_regex(df):
    '''
    general regex cleaning steps
    '''
    # read in regex_patterns.json
    regex_patterns = pd.read_json('regex_patterns_lukas.json').transpose().rename(columns={0:'new_value'})
    for col in df.columns:
        # remove html tags
        df[col] = df[col].apply(lambda x: re.sub('<.*?>','',x) if type(x) == str else x)
        # remove punctuation
        df[col] = df[col].apply(lambda x: re.sub('[^\w\s]','',x) if type(x) == str else x)
        # remove observed patterns listed "regex_patterns.json"
        for pattern in regex_patterns.index:
            df[col] = df[col].apply(lambda x: re.sub(f'{pattern}',f'{regex_patterns.loc[pattern]["new_value"]}',x) if type(x) == str else x)
    return df

In [29]:
df = general_regex(lowercase(train))
df

,description,host_about,unlisted
index,,,
1,this is a shared mixed room in our hostel with...,alojamento local registro 20835al,0
2,o meu espaço fica perto de parque eduardo vii ...,i am friendly host and i will try to always be...,1
3,trafarias house is a cozy and familiar villa w...,i am a social person liking to communicate rea...,1
4,apartamento charmoso no chiado entre o largo d...,hello\ni m portuguese and i love to meet peopl...,0
5,joli appartement en bordure de mer 2 min a pi...,nous sommes une famille avec deux enfants de 1...,0
...,...,...,...
12492,cats by bairro altothis cozy apartment in lisb...,travelling is one of my favorite hobbies ive a...,0
12493,beautifully located in the heart of lisbons hi...,founded by travel enthusiasts just like you bn...,0
12494,enjoy breakfast in the sleek kitchen with its ...,im from portugal and i love to dance and to tr...,0


In [5]:
regex_patterns = pd.read_json('regex_patterns_lukas.json').transpose().rename(columns={0:'new_value'})

In [6]:
regex_patterns

,new_value
_x005f_x000d_,
_x000d_,
m2,squaremeter
sm,squaremeter
sqm,squaremeter
m²,squaremeter
license[ number]+[0-9a-z]+,
(\b\w)\1+\b,
